In [1]:
import os
import time
import string
import pathlib
import requests
import operator
import functools
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from autoscraper import AutoScraper
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from nltk.sentiment import SentimentIntensityAnalyzer
from selenium.webdriver.support.ui import WebDriverWait
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
# checking whether it is safe to scrape this website
r = requests.get("https://coinmarketcap.com/currencies/bitcoin/historical-data/")
r.status_code

200

# Scraping function for coinmarket

In [3]:
def scrape_crypto(crypto, period):
    
    url = f"https://coinmarketcap.com/currencies/{crypto}/historical-data/"
    driver = webdriver.Chrome(executable_path="/Users/siddheshshaji/Downloads/chromedriver")
    driver.get(url)
    time.sleep(3)
    driver.execute_script("window.scrollTo(0,900)")
    
    try:
        for i in range(12*period-1):
            time.sleep(1)
            button = driver.find_element_by_xpath('//button[@class="x0o17e-0 DChGS"]')
            driver.execute_script("arguments[0].scrollIntoView();", button)
            button.send_keys(Keys.UP,Keys.UP,Keys.UP,Keys.UP,Keys.UP,Keys.UP)
            time.sleep(1)
            button.click()
    except:
        pass

    html = driver.page_source
    df = pd.read_html(html)[0]
    df = df.replace('\$','',regex=True)
    driver.close()
    print("{}'s data scraped for the last {} year(s) successfully.".format(crypto, period))
    
    return df

In [4]:
cryptos = ['bitcoin', 'ethereum', 'cardano', 'tether', 'binance-coin']
period = 2
groups = dict()

start = time.time()
for i in cryptos:
    groups[i] = scrape_crypto(i,period)
end = time.time()

print("\nTook about: {} mins".format((end - start)/60))

for i in groups:
    groups[i].to_csv(f'{i}_data.csv', index=False)

bitcoin's data scraped for the last 2 year(s) successfully.
ethereum's data scraped for the last 2 year(s) successfully.
cardano's data scraped for the last 2 year(s) successfully.
tether's data scraped for the last 2 year(s) successfully.
binance-coin's data scraped for the last 2 year(s) successfully.

Took about: 4.081303318341573 mins


# Scraping function for lexis-nexis

In [5]:
# checking whether it is safe to scrape this website
r = requests.get('https://advance.lexis.com/')
r.status_code

200

In [ ]:
url = 'https://advance.lexis.com/'
driver = webdriver.Chrome(executable_path="/Users/siddheshshaji/Downloads/chromedriver")
driver.get(url)

time.sleep(3)

# login

uid = driver.find_element_by_id("userid")
uid.send_keys('lolhaha199@gmail.com')

pwd = driver.find_element_by_id("password")
pwd.send_keys('Hahahalol123!')
pwd.submit() 

time.sleep(1)

# search for the term

search = driver.find_element_by_xpath('//*[@id="searchTerms"]')
search.send_keys('bitcoin')
search.send_keys(Keys.RETURN)

time.sleep(2)

# Dates dropdown

button = driver.find_element_by_xpath('//*[@class="dropdown-toggle fixedWidth"]')
time.sleep(1)
button.click()
time.sleep(1)
button = driver.find_element_by_xpath('//*[@data-value="datedescending"]')
time.sleep(1)
button.click()

# negative news filter

button = driver.find_element_by_xpath('//*[@id="podfiltersbuttonnegativenews"]')
time.sleep(2)
button.click()
time.sleep(3)
negnews1 = driver.find_element_by_xpath('//*[@id="refine"]/ul[10]/li[2]/label')
time.sleep(1)
actions = ActionChains(driver)
actions.move_to_element(negnews1).perform()

negnews1.click()

time.sleep(2)

# news source type filter

button2 = driver.find_element_by_xpath('//*[@id="podfiltersbuttonsource"]')
time.sleep(3)
button2.click()

button2.send_keys(Keys.DOWN,Keys.DOWN,Keys.DOWN)
time.sleep(3)
news_source = driver.find_element_by_xpath('//*[@id="refine"]/ul[5]/li[1]/label')

time.sleep(2)
actions = ActionChains(driver)
actions.move_to_element(news_source).perform()

news_source.click()

titles_ = []
dates_ = []

time.sleep(4)

for j in range(5):
    try:
        # titles of the news articles
        titles = driver.find_elements_by_xpath('//a[@data-action="title"]')
        titles = driver.find_elements_by_xpath('//a[@data-action="title"]')
        titles_.append([i.text for i in titles])
        time.sleep(1)

        # dates of the news articles(declared twice in order to solve the stalelement exception)
        dates = driver.find_elements_by_xpath('//a[@data-type="nexis-props-date"]')
        dates_.append([i.text for i in dates])
        time.sleep(2)

        # pagination
        pagin = driver.find_element_by_link_text('{}'.format(j+2))
        time.sleep(1)
        actions = ActionChains(driver)
        actions.move_to_element(pagin).perform()
        pagin.click()
    except:
        continue
    
titles = functools.reduce(operator.iconcat, titles_, [])
dates = functools.reduce(operator.iconcat, dates_, [])

driver.close()

# Sentiment analysis on the news

In [30]:
sid = SentimentIntensityAnalyzer()

In [31]:
for i in range(len(titles)):
    print(sid.polarity_scores(titles[i]),titles[i])

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} Russian President Vladimir Putin: Bitcoin Has 'Right to Exist'
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} Following The Decade-Long IMF Playbook
{'neg': 0.317, 'neu': 0.549, 'pos': 0.134, 'compound': -0.4588} ONECOIN INVESTORS FROM FAILED NEW YORK LAWSUIT JOIN CLAIMANTS IN LONDON & BRUSSELS
{'neg': 0.0, 'neu': 0.855, 'pos': 0.145, 'compound': 0.296} Insider Selling: HIVE Blockchain Technologies Ltd. (CVE:HIVE) Senior Officer Sells 3,000 Shares of Stock
{'neg': 0.478, 'neu': 0.522, 'pos': 0.0, 'compound': -0.6705} Unemployment claims fall to lowest level of pandemic
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} Vident Investment Advisory LLC Makes New Investment in Bitfarms Ltd. (NASDAQ:BITF)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} Vident Investment Advisory LLC Purchases New Position in Bitfarms Ltd. (NASDAQ:BITF)
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0} How Decentralized Identifiers And

In [32]:
df = pd.DataFrame(
    {'dates': dates,
     'news': titles
    })
df = df.drop_duplicates('news')

In [33]:
df

,dates,news
0,"Oct 14, 2021",Russian President Vladimir Putin: Bitcoin Has ...
1,"Oct 14, 2021",Following The Decade-Long IMF Playbook
2,"Oct 14, 2021",ONECOIN INVESTORS FROM FAILED NEW YORK LAWSUIT...
3,"Oct 14, 2021",Insider Selling: HIVE Blockchain Technologies ...
4,"Oct 14, 2021",Unemployment claims fall to lowest level of pa...
5,"Oct 14, 2021",Vident Investment Advisory LLC Makes New Inves...
6,"Oct 14, 2021",Vident Investment Advisory LLC Purchases New P...
7,"Oct 13, 2021",How Decentralized Identifiers And Bitcoin Fix ...
8,"Oct 13, 2021",The Impending Binance Arbitration: a Primer on...
9,"Oct 13, 2021",Gates of Vienna News Feed 10/12/2021
